In [1]:
%%capture
# Capture because braindecode module prints a lot :S
from pyFiles.training import *
from pyFiles.models import *
from pyFiles.dataprep_down import *
from pyFiles.dataprep2_down import *
#
import torch
from torch import nn
from torch.nn import functional as F
#
from pyFiles.losses import InfoNceLoss, VICReg, BarlowTwins

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [3]:
#PREP DENUEVO
# prep("C:\\Users\\TheSy\\Desktop\\dataverse_files", mode = "per_channel", save = True, save_dir = "data_down")

In [4]:
# spliter = DFSpliter(save = True, mode= "down_ch")
# spliter("down_prep_channels.csv",".")

In [5]:
transforms = Augmentations(5, False, AUGMENTATIONS)

In [6]:
p_size = int(1216/2) # Max = 19*64 = 1216
class CustomEEGDataset(Dataset):
    def __init__(self, csv_file , root_dir , transform = None, ):

        try:
            self.loc_df = pd.read_csv(os.path.join(root_dir,csv_file)).drop(labels="Unnamed: 0", axis = 1)
        except:
            self.loc_df = pd.read_csv(os.path.join(root_dir,csv_file))

        self.transform = transform
        self.root_dir = root_dir

    def __len__(self,):
        return len(self.loc_df)//p_size
    

    def __getitem__(self, idx):
        exam_shape = (p_size, 1, 1000)
        # print(val_shape)
            # return ch, diagnostic
        new_exam = torch.ones(exam_shape)
        # for i in range(len(self.loc_df)):
        for j in range(p_size):
            eeg_file = os.path.join(self.root_dir,
                            self.loc_df.iloc[idx*p_size + j, 2])
            ch, diagnostic = torch.load(eeg_file)
        
            if self.transform is not None:
                ch = self.transform(ch)
            new_exam[j] = ch
        
        return new_exam, diagnostic

In [7]:
train_dataset = CustomEEGDataset("down_ch_train_feats.csv", root_dir = "", transform = transforms)
val_dataset = CustomEEGDataset("down_ch_val_feats.csv", root_dir = "")

In [8]:
# trainloader = DataLoader(train_dataset, batch_size = 1, shuffle = True, num_workers = 0)
trainloader = DataLoader(train_dataset, batch_size = 4, shuffle = True, num_workers = 0)
valloader = DataLoader(val_dataset, batch_size = 12, shuffle = False, num_workers = 0)

In [10]:
model = SleepStagerSingle(n_chans = 1, pretext = False, p_size= p_size)
enc, fc = torch.load("params\singles\VR\\attempt3\\bestEncoderParamssingle_vr.pt")
model.encoder.load_state_dict(enc)

criterion = nn.CrossEntropyLoss()

# for param in model.encoder.feature_extractor[0].parameters():
#     param.requires_grad = False

optimizer = torch.optim.Adam(
    [
        {"params": model.encoder.parameters(), "lr": 1e-6},
        {"params": model.final.parameters(), "lr": 1e-4},
    ],
    lr=5e-4,
) 
downtrain(model, 30, trainloader, valloader, criterion, optimizer, state = None, name = "single_VR")

Epoch nro 1/30


KeyboardInterrupt: 